In [11]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn import preprocessing 
from collections import namedtuple
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as offline
from plotly.subplots import make_subplots

In [12]:
#loading in the df
df = pd.read_csv("data\\2011-2020.csv")

In [13]:
#filtering df
df = df.dropna()

# df[['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
# 'September', 'October', 'November', 'December']] = df[['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
# 'September', 'October', 'November', 'December']].apply(lambda x: x/100)



In [14]:
#creating the correlation dataframe
correlationdf = df[['LATITUDE', 'LONGITUDE', 'STNELEV', 'Year', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
'September', 'October', 'November', 'December']]
correlationdf.corr(method="pearson")


,LATITUDE,LONGITUDE,STNELEV,Year,January,February,March,April,May,June,July,August,September,October,November,December
LATITUDE,1.000000,-0.320991,0.037920,-0.019059,-0.716844,-0.686624,-0.623176,-0.479858,-0.223185,0.051941,0.206645,0.126519,-0.144698,-0.451302,-0.622778,-0.686914
LONGITUDE,-0.320991,1.000000,-0.287447,-0.027336,0.165444,0.173891,0.164519,0.177534,0.086269,-0.055287,-0.145025,-0.125189,-0.044351,0.104209,0.172963,0.187423
STNELEV,0.037920,-0.287447,1.000000,0.022515,-0.167810,-0.187112,-0.200427,-0.285772,-0.342213,-0.268384,-0.216726,-0.223027,-0.266419,-0.289010,-0.245495,-0.246206
Year,-0.019059,-0.027336,0.022515,1.000000,0.043880,0.046035,0.022776,0.005468,0.015052,0.018183,0.011825,0.013516,0.023621,0.001448,0.020356,0.024809
January,-0.716844,0.165444,-0.167810,0.043880,1.000000,0.963475,0.929647,0.830398,0.625759,0.378778,0.250314,0.354972,0.608065,0.843287,0.942169,0.958706
February,-0.686624,0.173891,-0.187112,0.046035,0.963475,1.000000,0.950733,0.863532,0.679036,0.452504,0.318555,0.414540,0.654099,0.868245,0.937023,0.936606
March,-0.623176,0.164519,-0.200427,0.022776,0.929647,0.950733,1.000000,0.932281,0.787119,0.593486,0.470254,0.550610,0.749362,0.907586,0.942281,0.915909
April,-0.479858,0.177534,-0.285772,0.005468,0.830398,0.863532,0.932281,1.000000,0.909855,0.762474,0.645423,0.708354,0.859967,0.940244,0.898842,0.845635
May,-0.223185,0.086269,-0.342213,0.015052,0.625759,0.679036,0.787119,0.909855,1.000000,0.911600,0.829033,0.862214,0.923454,0.879310,0.747293,0.670073
June,0.051941,-0.055287,-0.268384,0.018183,0.378778,0.452504,0.593486,0.762474,0.911600,1.000000,0.947964,0.946915,0.908608,0.748052,0.542973,0.424373


In [15]:
#we're going to have to create a model for each month due to the nature of the dataset
import sklearn

#fancy tuple for output of create_train_test function
ModelSets = namedtuple('testsets', ["Month", "X_train","y_train","X_test","y_test"])

#months of year
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
    'September', 'October', 'November', 'December']

#this function splits the train and test set depending on the month we're creating the model for
def create_train_test(train: pd.DataFrame, test: pd.DataFrame, month: str) -> ModelSets: 
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
    'September', 'October', 'November', 'December']
    X_train = train.drop(columns=months)
    y_train = train[month]

    X_test = test.drop(columns=months)
    y_test = test[month]


    outset = ModelSets(month, X_train,y_train,X_test,y_test)

    return outset

# Prints out the statistics we want
def print_statistics(lr: sklearn.linear_model.LinearRegression, modelsets: ModelSets):
    print("----")
    print("LinearRegression model for " + modelsets.Month)
    print("R^2 Score for training")
    acurtrain = lr.score(modelsets.X_train, modelsets.y_train)
    print(acurtrain)
    print("R^2 Score for testing")
    acurtest = lr.score(modelsets.X_test, modelsets.y_test)
    print(acurtest)
    print("----")

#creates a model specfic to the month
def create_model(df: pd.DataFrame, month: str) -> sklearn.linear_model.LinearRegression:
    percentfortrain = .9
    train = df.sample(frac=percentfortrain)
    test = df.drop(index=train.index)

    modelsets = create_train_test(train,test,month)
    
    lr = LinearRegression(fit_intercept=False)

    lr.fit(modelsets.X_train, modelsets.y_train)
    print_statistics(lr, modelsets)

    return lr


In [16]:
#creating the linear regression model for each month and savin ghtem to a dictionary

lrmodels = {}

for i in months:
    lrmodels[i] = create_model(correlationdf, i)

----
LinearRegression model for January
R^2 Score for training
0.5485670966436837
R^2 Score for testing
0.5305234788874413
----
----
LinearRegression model for February
R^2 Score for training
0.5071182479683773
R^2 Score for testing
0.5121362243365063
----
----
LinearRegression model for March
R^2 Score for training
0.4296545165193333
R^2 Score for testing
0.41658681580680246
----
----
LinearRegression model for April
R^2 Score for training
0.30849522145663144
R^2 Score for testing
0.27365609462891727
----
----
LinearRegression model for May
R^2 Score for training
0.16735698195283977
R^2 Score for testing
0.1806521391787037
----
----
LinearRegression model for June
R^2 Score for training
0.09247721793226882
R^2 Score for testing
0.08406378275597204
----
----
LinearRegression model for July
R^2 Score for training
0.11662509291438505
R^2 Score for testing
0.12433655787550457
----
----
LinearRegression model for August
R^2 Score for training
0.09628876479355475
R^2 Score for testing
0.078

In [17]:
#doing linear regression modelling on adjusted to actual celcius 
normaldf = correlationdf

normaldf[['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
'September', 'October', 'November', 'December']] = df[['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
'September', 'October', 'November', 'December']].apply(lambda x: x/100)



normlrmodels = {}

for i in months:
    normlrmodels[i] = create_model(normaldf, i)

normaldf.corr(method="pearson")


C:\Users\alpagar\AppData\Local\Temp\ipykernel_7512\1358372081.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



----
LinearRegression model for January
R^2 Score for training
0.5441049446643991
R^2 Score for testing
0.5708187936390129
----
----
LinearRegression model for February
R^2 Score for training
0.5046449973753208
R^2 Score for testing
0.5344882644723303
----
----
LinearRegression model for March
R^2 Score for training
0.4269243656516969
R^2 Score for testing
0.4411113404614807
----
----
LinearRegression model for April
R^2 Score for training
0.3060160443081309
R^2 Score for testing
0.29626932938210526
----
----
LinearRegression model for May
R^2 Score for training
0.16921091713761793
R^2 Score for testing
0.16441954285650617
----
----
LinearRegression model for June
R^2 Score for training
0.09213016878727309
R^2 Score for testing
0.08699110119206344
----
----
LinearRegression model for July
R^2 Score for training
0.11793754356407837
R^2 Score for testing
0.11186553531072385
----
----
LinearRegression model for August
R^2 Score for training
0.09526917816081037
R^2 Score for testing
0.0877

,LATITUDE,LONGITUDE,STNELEV,Year,January,February,March,April,May,June,July,August,September,October,November,December
LATITUDE,1.000000,-0.320991,0.037920,-0.019059,-0.716844,-0.686624,-0.623176,-0.479858,-0.223185,0.051941,0.206645,0.126519,-0.144698,-0.451302,-0.622778,-0.686914
LONGITUDE,-0.320991,1.000000,-0.287447,-0.027336,0.165444,0.173891,0.164519,0.177534,0.086269,-0.055287,-0.145025,-0.125189,-0.044351,0.104209,0.172963,0.187423
STNELEV,0.037920,-0.287447,1.000000,0.022515,-0.167810,-0.187112,-0.200427,-0.285772,-0.342213,-0.268384,-0.216726,-0.223027,-0.266419,-0.289010,-0.245495,-0.246206
Year,-0.019059,-0.027336,0.022515,1.000000,0.043880,0.046035,0.022776,0.005468,0.015052,0.018183,0.011825,0.013516,0.023621,0.001448,0.020356,0.024809
January,-0.716844,0.165444,-0.167810,0.043880,1.000000,0.963475,0.929647,0.830398,0.625759,0.378778,0.250314,0.354972,0.608065,0.843287,0.942169,0.958706
February,-0.686624,0.173891,-0.187112,0.046035,0.963475,1.000000,0.950733,0.863532,0.679036,0.452504,0.318555,0.414540,0.654099,0.868245,0.937023,0.936606
March,-0.623176,0.164519,-0.200427,0.022776,0.929647,0.950733,1.000000,0.932281,0.787119,0.593486,0.470254,0.550610,0.749362,0.907586,0.942281,0.915909
April,-0.479858,0.177534,-0.285772,0.005468,0.830398,0.863532,0.932281,1.000000,0.909855,0.762474,0.645423,0.708354,0.859967,0.940244,0.898842,0.845635
May,-0.223185,0.086269,-0.342213,0.015052,0.625759,0.679036,0.787119,0.909855,1.000000,0.911600,0.829033,0.862214,0.923454,0.879310,0.747293,0.670073
June,0.051941,-0.055287,-0.268384,0.018183,0.378778,0.452504,0.593486,0.762474,0.911600,1.000000,0.947964,0.946915,0.908608,0.748052,0.542973,0.424373


In [18]:
#doing linear regression modeling on only upper hemisphere
upperhemispheredf = correlationdf[correlationdf["LATITUDE"] > 0]

uphemlrmodels = {}

for i in months:
    uphemlrmodels[i] = create_model(upperhemispheredf, i)


upperhemispheredf.corr(method='pearson')


----
LinearRegression model for January
R^2 Score for training
0.6322437891415309
R^2 Score for testing
0.6417466584175704
----
----
LinearRegression model for February
R^2 Score for training
0.6526577389544889
R^2 Score for testing
0.6563013024948758
----
----
LinearRegression model for March
R^2 Score for training
0.7328325319297768
R^2 Score for testing
0.736288799778434
----
----
LinearRegression model for April
R^2 Score for training
0.7961490172708443
R^2 Score for testing
0.8007756816537019
----
----
LinearRegression model for May
R^2 Score for training
0.7893274466571145
R^2 Score for testing
0.7799402545052656
----
----
LinearRegression model for June
R^2 Score for training
0.693598752894578
R^2 Score for testing
0.6928546381389511
----
----
LinearRegression model for July
R^2 Score for training
0.6290945551848446
R^2 Score for testing
0.6300324400836246
----
----
LinearRegression model for August
R^2 Score for training
0.6947366728974398
R^2 Score for testing
0.69088907077790

,LATITUDE,LONGITUDE,STNELEV,Year,January,February,March,April,May,June,July,August,September,October,November,December
LATITUDE,1.000000,0.090732,-0.101384,-0.025983,-0.765705,-0.774759,-0.820821,-0.816768,-0.767705,-0.729525,-0.701111,-0.747869,-0.810797,-0.826247,-0.794013,-0.751248
LONGITUDE,0.090732,1.000000,-0.287266,-0.040380,-0.088162,-0.050714,-0.017098,0.086762,0.109435,0.072408,0.023689,0.017773,-0.017492,0.009206,-0.006349,-0.036050
STNELEV,-0.101384,-0.287266,1.000000,0.028209,-0.122588,-0.144852,-0.158995,-0.269617,-0.362120,-0.324360,-0.297487,-0.289098,-0.290418,-0.281069,-0.222011,-0.225639
Year,-0.025983,-0.040380,0.028209,1.000000,0.048810,0.051785,0.021066,0.001119,0.016289,0.024921,0.019300,0.024819,0.030485,-0.002259,0.019576,0.023970
January,-0.765705,-0.088162,-0.122588,0.048810,1.000000,0.948718,0.921159,0.849204,0.736437,0.628697,0.590981,0.677133,0.790824,0.885873,0.935242,0.940516
February,-0.774759,-0.050714,-0.144852,0.051785,0.948718,1.000000,0.941148,0.874118,0.773938,0.685446,0.636579,0.709176,0.812668,0.897166,0.920452,0.909302
March,-0.820821,-0.017098,-0.158995,0.021066,0.921159,0.941148,1.000000,0.931384,0.848733,0.780451,0.739403,0.790421,0.857361,0.910159,0.923966,0.896045
April,-0.816768,0.086762,-0.269617,0.001119,0.849204,0.874118,0.931384,1.000000,0.931011,0.874817,0.819931,0.854873,0.903673,0.926206,0.893823,0.857019
May,-0.767705,0.109435,-0.362120,0.016289,0.736437,0.773938,0.848733,0.931011,1.000000,0.931267,0.879691,0.893369,0.906991,0.882147,0.807442,0.773595
June,-0.729525,0.072408,-0.324360,0.024921,0.628697,0.685446,0.780451,0.874817,0.931267,1.000000,0.935309,0.924350,0.897019,0.832052,0.724157,0.660493


In [19]:
#doing linear regression modeling on only lower hemisphere
lowerhemispheredf = correlationdf[correlationdf["LATITUDE"] < 0]

lowhemlrmodels = {}

for i in months:
    lowhemlrmodels[i] = create_model(lowerhemispheredf, i)


lowerhemispheredf.corr(method='pearson')


----
LinearRegression model for January
R^2 Score for training
0.7048967177864038
R^2 Score for testing
0.729548418880722
----
----
LinearRegression model for February
R^2 Score for training
0.7267036153720074
R^2 Score for testing
0.713552350667404
----
----
LinearRegression model for March
R^2 Score for training
0.760888695349724
R^2 Score for testing
0.7363933921323009
----
----
LinearRegression model for April
R^2 Score for training
0.8057162083321431
R^2 Score for testing
0.8174585618521648
----
----
LinearRegression model for May
R^2 Score for training
0.8505409769107907
R^2 Score for testing
0.8576956765247045
----
----
LinearRegression model for June
R^2 Score for training
0.8682587889685279
R^2 Score for testing
0.8677209592261079
----
----
LinearRegression model for July
R^2 Score for training
0.8704807599558209
R^2 Score for testing
0.8728230988498653
----
----
LinearRegression model for August
R^2 Score for training
0.8627420720857765
R^2 Score for testing
0.882940643527174

,LATITUDE,LONGITUDE,STNELEV,Year,January,February,March,April,May,June,July,August,September,October,November,December
LATITUDE,1.000000,-0.064701,-0.085202,-0.009738,0.804434,0.813813,0.836939,0.871703,0.900122,0.912113,0.913611,0.912556,0.902111,0.889545,0.872150,0.840013
LONGITUDE,-0.064701,1.000000,-0.159235,0.022853,0.102353,0.086009,0.069896,0.031024,-0.004813,-0.018073,-0.014500,-0.034764,-0.011080,0.018241,0.050221,0.067374
STNELEV,-0.085202,-0.159235,1.000000,-0.017086,-0.281202,-0.298166,-0.294552,-0.283465,-0.277106,-0.267037,-0.266271,-0.255748,-0.249842,-0.250584,-0.254309,-0.264623
Year,-0.009738,0.022853,-0.017086,1.000000,0.024062,0.011987,0.022667,0.017855,0.009688,0.006993,0.009174,-0.010648,-0.000796,0.010314,0.009541,0.023002
January,0.804434,0.102353,-0.281202,0.024062,1.000000,0.986387,0.970902,0.951542,0.918294,0.895865,0.896772,0.909982,0.934409,0.956679,0.971514,0.983003
February,0.813813,0.086009,-0.298166,0.011987,0.986387,1.000000,0.987623,0.971918,0.943877,0.923331,0.923016,0.934104,0.952489,0.968458,0.973522,0.978113
March,0.836939,0.069896,-0.294552,0.022667,0.970902,0.987623,1.000000,0.988050,0.968520,0.951319,0.951105,0.957410,0.970573,0.976350,0.973333,0.966871
April,0.871703,0.031024,-0.283465,0.017855,0.951542,0.971918,0.988050,1.000000,0.986633,0.975822,0.976008,0.978363,0.983824,0.981173,0.970972,0.956565
May,0.900122,-0.004813,-0.277106,0.009688,0.918294,0.943877,0.968520,0.986633,1.000000,0.992363,0.991134,0.990252,0.986711,0.975585,0.954895,0.932204
June,0.912113,-0.018073,-0.267037,0.006993,0.895865,0.923331,0.951319,0.975822,0.992363,1.000000,0.994731,0.991227,0.982422,0.965675,0.942335,0.912808


In [23]:
hemispheredf = correlationdf
#creating a new column where hemisphere is associated with a value of 1 or 0
hemispheredf["hemisphere"] = correlationdf["LATITUDE"].apply(lambda x: int(x < 0))

hemlrmodels = {}

for i in months:
    hemlrmodels[i] = create_model(hemispheredf, i)


hemispheredf.corr(method='pearson')


C:\Users\alpagar\AppData\Local\Temp\ipykernel_7512\3591820173.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



----
LinearRegression model for January
R^2 Score for training
0.5722959108074611
R^2 Score for testing
0.5824555187998077
----
----
LinearRegression model for February
R^2 Score for training
0.5442760467138161
R^2 Score for testing
0.572829958208981
----
----
LinearRegression model for March
R^2 Score for training
0.5004356668137167
R^2 Score for testing
0.46387167528298934
----
----
LinearRegression model for April
R^2 Score for training
0.4210131855660906
R^2 Score for testing
0.39283460747873267
----
----
LinearRegression model for May
R^2 Score for training
0.33324822032476253
R^2 Score for testing
0.3410232971790973
----
----
LinearRegression model for June
R^2 Score for training
0.29033818226786967
R^2 Score for testing
0.294519261463131
----
----
LinearRegression model for July
R^2 Score for training
0.30936965471811995
R^2 Score for testing
0.2907326497264957
----
----
LinearRegression model for August
R^2 Score for training
0.2896424218817517
R^2 Score for testing
0.286866010

,LATITUDE,LONGITUDE,STNELEV,Year,January,February,March,April,May,June,July,August,September,October,November,December,hemisphere
LATITUDE,1.000000,-0.320991,0.037920,-0.019059,-0.716844,-0.686624,-0.623176,-0.479858,-0.223185,0.051941,0.206645,0.126519,-0.144698,-0.451302,-0.622778,-0.686914,-0.870558
LONGITUDE,-0.320991,1.000000,-0.287447,-0.027336,0.165444,0.173891,0.164519,0.177534,0.086269,-0.055287,-0.145025,-0.125189,-0.044351,0.104209,0.172963,0.187423,0.404114
STNELEV,0.037920,-0.287447,1.000000,0.022515,-0.167810,-0.187112,-0.200427,-0.285772,-0.342213,-0.268384,-0.216726,-0.223027,-0.266419,-0.289010,-0.245495,-0.246206,-0.099190
Year,-0.019059,-0.027336,0.022515,1.000000,0.043880,0.046035,0.022776,0.005468,0.015052,0.018183,0.011825,0.013516,0.023621,0.001448,0.020356,0.024809,0.008419
January,-0.716844,0.165444,-0.167810,0.043880,1.000000,0.963475,0.929647,0.830398,0.625759,0.378778,0.250314,0.354972,0.608065,0.843287,0.942169,0.958706,0.542999
February,-0.686624,0.173891,-0.187112,0.046035,0.963475,1.000000,0.950733,0.863532,0.679036,0.452504,0.318555,0.414540,0.654099,0.868245,0.937023,0.936606,0.502576
March,-0.623176,0.164519,-0.200427,0.022776,0.929647,0.950733,1.000000,0.932281,0.787119,0.593486,0.470254,0.550610,0.749362,0.907586,0.942281,0.915909,0.419554
April,-0.479858,0.177534,-0.285772,0.005468,0.830398,0.863532,0.932281,1.000000,0.909855,0.762474,0.645423,0.708354,0.859967,0.940244,0.898842,0.845635,0.269713
May,-0.223185,0.086269,-0.342213,0.015052,0.625759,0.679036,0.787119,0.909855,1.000000,0.911600,0.829033,0.862214,0.923454,0.879310,0.747293,0.670073,0.014010
June,0.051941,-0.055287,-0.268384,0.018183,0.378778,0.452504,0.593486,0.762474,0.911600,1.000000,0.947964,0.946915,0.908608,0.748052,0.542973,0.424373,-0.253112


In [ ]:
#function to create plot for each month
def create_LR_plot(lr: sklearn.linear_model.LinearRegression,  modelsets: ModelSets, features: list):
    lr_predic = lr.predict(modelsets.X_test)
    for i in features: 
        
        fig1 = go.Scatter(x=modelsets.X_test[i].to_list(), y=modelsets.y_test.to_list(), name="actual", mode="markers")
        fig2 = go.Scatter(x=modelsets.X_test[i].to_list(), y=lr_predic, name="prediction", mode="markers")

        titlestr = i + " prediction for month of " + modelsets.Month + "."
        fig = make_subplots()
        fig.add_trace(fig1)
        fig.add_trace(fig2)
        fig.update_layout(xaxis_title=i, yaxis_title="Temperature (C)", title=titlestr)
        fig.show()

        figfilename = "data//modelvisulizations//" + i + "_"  + modelsets.Month
        offline.plot(fig, filename=figfilename)


train = correlationdf.sample(frac=.9)
test = correlationdf.drop(index=train.index)
features = ['LATITUDE', 'LONGITUDE', 'STNELEV', 'Year']

for i in months:
    modelsets = create_train_test(train,test,i)
    create_LR_plot(lrmodels[i], modelsets, features )
